In [12]:
import os
import json
from datetime import datetime

def parse_time(time_str):
    """
    Parse a time string of the form "15_50_39" or "15_42" into (hour, minute, second).
    If seconds are missing, assume 0.
    """
    parts = time_str.split('_')
    if len(parts) == 3:
        hour, minute, second = map(int, parts)
    elif len(parts) == 2:
        hour, minute = map(int, parts)
        second = 0
    else:
        raise ValueError("Unexpected time format: " + time_str)
    return hour, minute, second

def time_to_seconds(hour, minute, second):
    return hour * 3600 + minute * 60 + second

def extract_date_time_from_mini(mini_path):
    """
    Given a mini_path, extract the date folder and the time folder.
    Expected mini_path example:
    /data/big_rim/rsync_dcc_sum/Oct3V1mini_sorted/20241002-pmc-r2/customEntValHere/2024_10_25/15_50_39/
    
    Returns:
        date_folder (str): e.g. "2024_10_25"
        time_folder (str): e.g. "15_50_39"
    """
    # Normalize the path and split by the OS separator.
    parts = os.path.normpath(mini_path).split(os.sep)
    # Assuming the last two meaningful parts are the date folder and the time folder.
    # (Adjust the index if your path structure is different.)
    date_folder = parts[-2]
    time_folder = parts[-1]
    return date_folder, time_folder

def find_best_rec_path(mini_path, rec_base):
    """
    For a given mini_path, this function:
      1. Extracts the date and time.
      2. Looks into the rec_base/date_folder directory.
      3. Scans all candidate rec files (or directories) for one whose embedded time is closest to the mini_path time.
    
    Returns the full rec_path if found, or None.
    """
    date_folder, mini_time_str = extract_date_time_from_mini(mini_path)
    mini_hour, mini_minute, mini_second = parse_time(mini_time_str)
    mini_total_seconds = time_to_seconds(mini_hour, mini_minute, mini_second)
    
    # Build the directory path in which to look for rec files.
    rec_date_dir = os.path.join(rec_base, date_folder)
    if not os.path.isdir(rec_date_dir):
        print(f"Date folder {rec_date_dir} not found for mini_path {mini_path}")
        return None

    candidates = os.listdir(rec_date_dir)
    best_match = None
    best_diff = None

    for candidate in candidates:
        # Assume candidate names are like "20241002PMCr2_15_42".
        # Split by underscore and join the last two tokens to get the time string.
        tokens = candidate.split('_')
        if len(tokens) < 2:
            continue
        rec_time_str = tokens[-2] + '_' + tokens[-1]
        try:
            rec_hour, rec_minute, rec_second = parse_time(rec_time_str)
        except ValueError:
            continue  # Skip if the candidate does not have a recognizable time format.
        rec_total_seconds = time_to_seconds(rec_hour, rec_minute, rec_second)
        diff = abs(mini_total_seconds - rec_total_seconds)
        if best_diff is None or diff < best_diff:
            best_diff = diff
            best_match = candidate

    if best_match:
        best_rec_path = os.path.join(rec_date_dir, best_match)
        return best_rec_path
    else:
        print(f"No matching rec_path found in {rec_date_dir} for mini_path {mini_path}")
        return None

def automate_matching(mini_paths_file, rec_base, output_file):
    """
    Reads a text file containing mini_paths (one per line), finds the best matching rec_path
    for each mini_path, and saves the resulting mapping as a JSON file.
    """
    mapping = {}
    with open(mini_paths_file, 'r') as f:
        mini_paths = [line.strip() for line in f if line.strip()]
    
    for mini_path in mini_paths:
        rec_path = find_best_rec_path(mini_path, rec_base)
        mapping[mini_path] = rec_path
        print(f"Mini: {mini_path}\nMatched Rec: {rec_path}\n")
    
    with open(output_file, 'w') as f:
        json.dump(mapping, f, indent=4)
    print(f"Mapping saved to {output_file}")

if __name__ == "__main__":
    # File containing mini_paths (one per line). For example:
    # /data/big_rim/rsync_dcc_sum/Oct3V1mini_sorted/20241002-pmc-r2/customEntValHere/2024_10_25/15_50_39/
    mini_paths_file = "path_control.txt"
    
    # Base directory where rec_path date folders are stored.
    rec_base = "/data/big_rim/rsync_dcc_sum/Oct3V1"
    
    # Output file for the mini-to-rec mapping.
    output_file = "mini_to_rec_mapping.json"
    
    automate_matching(mini_paths_file, rec_base, output_file)


Mini: /data/big_rim/rsync_dcc_sum/Oct3V1mini_sorted/20241002-pmc-r2/customEntValHere/2024_10_25/15_50_39/
Matched Rec: /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_25/20241002PMCr2_15_42

Mini: /data/big_rim/rsync_dcc_sum/Oct3V1mini_sorted/20241002-pmc-r2/customEntValHere/2024_10_25/17_10_00/
Matched Rec: /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_25/20241002PMCr2_17_05

Mapping saved to mini_to_rec_mapping.json


In [ ]:
# have not tested because prob need some checking if it exist, and some handling of what if it does not exist, etc. 

# import os
# import sys
# import json

# # Import your functions
# sys.path.append(os.path.abspath('../..'))
# from utlis.sync_utlis.mini_dannce_sync import sync_videos, align_miniscope_to_sixcam

# def get_rec_path_for_mini(mini_path, rec_base):
#     """
#     Get the best matching rec_path for a given mini_path.
#     """
#     return find_best_rec_path(mini_path, rec_base)

# # Define paths
# rec_base = "/data/big_rim/rsync_dcc_sum/Oct3V1"
# mini_path = "/data/big_rim/rsync_dcc_sum/Oct3V1mini_sorted/20241002-pmc-r2/customEntValHere/2024_10_25/15_50_39/"

# # Find the best rec_path dynamically
# rec_path = get_rec_path_for_mini(mini_path, rec_base)

# if rec_path:
#     print(f"Using matched rec_path: {rec_path}")

#     # Step 1: Dannce alignment
#     resultsss = sync_videos(
#         rec_path,
#         mini_path,
#         start_frame=0,
#         end_frame=200,
#         threshold_mini=15,
#         threshold_sixcam=3
#     )

#     align_miniscope_to_sixcam(resultsss, mini_path, rec_path)

#     # Step 2: Load alignment and process calcium data
#     from utlis.sync_utlis.mini_dannce_sync import load_aligneddannce_and_process_ca_data

#     df_merged_with_dF_F = load_aligneddannce_and_process_ca_data(rec_path, mini_path)
# else:
#     print(f"No matching rec_path found for {mini_path}")
